# The Jets Collection

The Jets collections

In [1]:
from func_adl_servicex_xaodr21 import SXDSAtlasxAODR21
import matplotlib.pyplot as plt
from config import rucio_zee_r21_mc, xAODLocalTyped, local_xaod_path
import awkward as ak

In [2]:
#ds = SXDSAtlasxAODR21(rucio_zee_r21_mc, backend='dev_xaod')
ds = xAODLocalTyped(local_xaod_path)

Found backend type matching "xaod". Matching by type is depreciated. Please switch to using the "name" keyword in your servicex.yaml file.


In [3]:
import logging
logging.basicConfig()
logging.getLogger("func_adl_xAOD.common.local_dataset").setLevel(level=logging.DEBUG)
from servicex import ignore_cache
with ignore_cache(): 
        jets = (ds
                .SelectMany(lambda e: e.Jets("AntiKt4EMTopoJets"))
                .Where(lambda j: (j.pt() / 1000) > 30)
                .Select(lambda j: j.pt() / 1000.0)
                .AsAwkwardArray('JetPt')
                .value())

DEBUG:func_adl_xAOD.common.local_dataset:Docker image and tag: atlas/analysisbase:21.2.197
DEBUG:func_adl_xAOD.common.local_dataset:Docker Output: 
DEBUG:func_adl_xAOD.common.local_dataset:  Configured GCC from: /opt/lcg/gcc/8.3.0-cebb0/x86_64-centos7/bin/gcc
DEBUG:func_adl_xAOD.common.local_dataset:  Configured AnalysisBase from: /usr/AnalysisBase/21.2.197/InstallArea/x86_64-centos7-gcc8-opt
DEBUG:func_adl_xAOD.common.local_dataset:  -- The C compiler identification is GNU 8.3.0
DEBUG:func_adl_xAOD.common.local_dataset:  -- The CXX compiler identification is GNU 8.3.0
DEBUG:func_adl_xAOD.common.local_dataset:  -- Detecting C compiler ABI info
DEBUG:func_adl_xAOD.common.local_dataset:  -- Detecting C compiler ABI info - done
DEBUG:func_adl_xAOD.common.local_dataset:  -- Check for working C compiler: /opt/lcg/gcc/8.3.0-cebb0/x86_64-centos7/bin/gcc - skipped
DEBUG:func_adl_xAOD.common.local_dataset:  -- Detecting C compile features
DEBUG:func_adl_xAOD.common.local_dataset:  -- Detecting 

In [ ]:
plt.hist(jets.JetPt, bins=100, range=(0, 100))
plt.xlabel('Jet $p_T$ [GeV]')
plt.ylabel('Number of jets')
_ = plt.title('Jet $p_T$ distribution for $Z\\rightarrow ee$ events')

## Jet Constituents

Jets are composed of `TopoClusters` in ATLAS, and the AntiKt algorithm normally clusters them into a jet. In R21 files, the `TopoClusters` are easily accessible.

In [ ]:
from servicex import ignore_cache
with ignore_cache():
    topo_clusters = (ds
                     .SelectMany(lambda e: e.Jets("AntiKt4EMTopoJets"))
                     .SelectMany(lambda j: j.getConstituents())
                     .Select(lambda tc: tc.pt())
                     .AsAwkwardArray('JetClusterPt')
                     .value()
                    )

In [ ]:
plt.hist(topo_clusters.JetClusterPt/1000.0, bins=100, range=(0, 20))
plt.xlabel('Jet Cluster $p_T$ [GeV]')
plt.ylabel('Number of jets')
_ = plt.title('Jet Cluster $p_T$ distribution for jets in $Z\\rightarrow ee$ events')

## Calibration

By default the jets we pulled from above are calibrated, and the best central value for the jet collection you request is returned. You can also fetch uncalibrated jets as well as the systematic error variations.

First, uncorrected jets:

In [ ]:
jets_raw = (ds
           .SelectMany(lambda e: e.Jets("AntiKt4EMTopoJets", calibrated=False))
           .Where(lambda j: (j.pt() / 1000) > 30)
           .Select(lambda j: j.pt() / 1000.0)
           .AsAwkwardArray('JetPt')
           .value())

## The Datamodel

The data model when this documentation was last built was:

In [ ]:
from func_adl_servicex_xaodr21.xAOD.jet_v1 import Jet_v1
help(Jet_v1)

In [ ]:
from func_adl_servicex_xaodr21.xAOD.jetconstituent import JetConstituent
help(JetConstituent)

## Further Information

* The [`xAOD::Jet_v1` C++ header file](https://gitlab.cern.ch/atlas/athena/-/blob/21.2/Event/xAOD/xAODJet/xAODJet/versions/Jet_v1.h) with all the inline documentation.
* The [`xAOD::JetConstituent` C++ header File](https://gitlab.cern.ch/atlas/athena/-/blob/21.2/Event/xAOD/xAODJet/xAODJet/JetConstituentVector.h) with all the inline documentation.